In [1]:
import pandas as pd

# Load files
ledger = pd.read_csv(r"C:\Users\HarithaNagamalla\Downloads\SRC_LDGR_Bloomberg_251125 1.csv")
statement = pd.read_csv(r"C:\Users\HarithaNagamalla\Downloads\SRC_STMT_Calypso_251125 1.csv")

# Parse dates properly - use format='mixed' to handle varying formats
ledger["Trade_Date"] = pd.to_datetime(ledger["Trade_Date"], format="mixed", dayfirst=True)
statement["Trade_Date"] = pd.to_datetime(statement["Trade_Date"], format="mixed", dayfirst=True)

# Normalize to date only for matching (remove time component)
ledger["Trade_Date"] = ledger["Trade_Date"].dt.date
statement["Trade_Date"] = statement["Trade_Date"].dt.date

ledger["source_type"] = "ledger"
statement["source_type"] = "statement"

# Normalize signage
def normalize_signage(x):
    return x.strip().upper()

ledger["Signage"] = ledger["Signage"].apply(normalize_signage)
statement["Signage"] = statement["Signage"].apply(normalize_signage)

# Create signed amount
def signed_amount(row):
    if row["Signage"] in ["DR", "D"]:
        return -row["Amount1"]
    elif row["Signage"] in ["CR", "C"]:
        return row["Amount1"]
    else:
        return 0

ledger["signed_amount"] = ledger.apply(signed_amount, axis=1)
statement["signed_amount"] = statement.apply(signed_amount, axis=1)

# For matching, use absolute amounts (signage differences are normal between systems)
ledger["abs_amount"] = ledger["signed_amount"].abs()
statement["abs_amount"] = statement["signed_amount"].abs()


In [2]:
# Debug: Check columns and data types
print("LEDGER COLUMNS:")
print(ledger.columns.tolist())
print("\nLedger dtypes:")
print(ledger.dtypes)
print("\nLedger shape:", ledger.shape)
print("\nFirst few ledger rows:")
print(ledger[["Trade_Date", "Currency", "Tran_code", "Quantity", "signed_amount"]].head())

print("\n" + "="*50)
print("STATEMENT COLUMNS:")
print(statement.columns.tolist())
print("\nStatement dtypes:")
print(statement.dtypes)
print("\nStatement shape:", statement.shape)
print("\nFirst few statement rows:")
print(statement[["Trade_Date", "Currency", "Tran_code", "Quantity", "signed_amount"]].head())

LEDGER COLUMNS:
['Source', 'ISIN_CUSIP', 'Trade_Date', 'Currency', 'Tran_code', 'Quantity', 'Amount1', 'Amount2', 'Signage', 'Ref1', 'Ref2', 'Trade_status', 'source_type', 'signed_amount', 'abs_amount']

Ledger dtypes:
Source            object
ISIN_CUSIP        object
Trade_Date        object
Currency          object
Tran_code         object
Quantity           int64
Amount1          float64
Amount2          float64
Signage           object
Ref1              object
Ref2              object
Trade_status      object
source_type       object
signed_amount    float64
abs_amount       float64
dtype: object

Ledger shape: (1003, 15)

First few ledger rows:
   Trade_Date Currency Tran_code  Quantity  signed_amount
0  2022-05-07      CAD       DIV       111      -25633.43
1  2025-05-27      CAD       BUY       896      -75545.67
2  2024-03-27      GBP       INT       197       77472.69
3  2024-09-20      GBP       BUY       288       84100.53
4  2020-01-09      EUR       BUY       429       503

In [3]:
import json

def calculate_confidence(ledger_row, statement_row):
    """
    Calculate confidence score based on match quality.
    Perfect match = 1.0, deductions for minor discrepancies.
    """
    confidence = 1.0
    
    # Check for exact matches on key fields
    if ledger_row["ISIN_CUSIP"] != statement_row["ISIN_CUSIP"]:
        confidence -= 0.15
    
    if ledger_row["Trade_Date"] != statement_row["Trade_Date"]:
        confidence -= 0.10
    
    if ledger_row["Currency"] != statement_row["Currency"]:
        confidence -= 0.20
    
    if ledger_row["Quantity"] != statement_row["Quantity"]:
        confidence -= 0.15
    
    # Check if amounts match (should match after abs normalization)
    if abs(ledger_row["signed_amount"] - statement_row["signed_amount"]) > 0.01:
        confidence -= 0.20
    
    # Check references
    if ledger_row["Ref1"] != statement_row["Ref1"]:
        confidence -= 0.05
    
    if ledger_row["Ref2"] != statement_row["Ref2"]:
        confidence -= 0.05
    
    # Ensure confidence is between 0 and 1
    return max(0.0, min(1.0, confidence))

def create_finetune_sample(ledger_row, statement_row):
    confidence = calculate_confidence(ledger_row, statement_row)
    
    user_prompt = f"""
Ledger Transaction:
{ledger_row.to_dict()}

Statement Transaction:
{statement_row.to_dict()}

Task:
Check if these transactions reconcile.
Return:
- matched (true/false)
- reason
- confidence (0-1)
"""

    assistant_response = {
        "matched": True,
        "ledger_source": ledger_row["Source"],
        "statement_source": statement_row["Source"],
        "confidence": round(confidence, 2),
        "reason": (
            "Amounts, currency, trade date, transaction code, quantity "
            "and references match across ledger and statement. "
            "Signage difference handled via accounting rules."
        )
    }

    return {
        "messages": [
            {"role": "system", "content": "You are a financial reconciliation expert."},
            {"role": "user", "content": user_prompt},
            {"role": "assistant", "content": json.dumps(assistant_response)}
        ]
    }


In [4]:
match_keys = [
    "Trade_Date",
    "Currency",
    "Tran_code",
    "Quantity",
    "Ref1",
    "Ref2",
    "abs_amount"  # Use absolute amount for matching
]

merged = ledger.merge(
    statement,
    on=match_keys,
    suffixes=("_ledger", "_statement"),
    how="inner"
)

print("Matched rows:", len(merged))


Matched rows: 903


In [5]:
training_data = []

for _, row in merged.iterrows():

    ledger_row = {
        "Source": row["Source_ledger"],
        "ISIN_CUSIP": row["ISIN_CUSIP_ledger"],
        "Trade_Date": str(row["Trade_Date"]),
        "Currency": row["Currency"],
        "Tran_code": row["Tran_code"],
        "Quantity": row["Quantity"],
        "signed_amount": row["signed_amount_ledger"],
        "Ref1": row["Ref1"],
        "Ref2": row["Ref2"],
        "Trade_status": row["Trade_status_ledger"]
    }

    statement_row = {
        "Source": row["Source_statement"],
        "ISIN_CUSIP": row["ISIN_CUSIP_statement"],
        "Trade_Date": str(row["Trade_Date"]),
        "Currency": row["Currency"],
        "Tran_code": row["Tran_code"],
        "Quantity": row["Quantity"],
        "signed_amount": row["signed_amount_statement"],
        "Ref1": row["Ref1"],
        "Ref2": row["Ref2"],
        "Trade_status": row["Trade_status_statement"]
    }

    training_data.append(
        create_finetune_sample(
            pd.Series(ledger_row),
            pd.Series(statement_row)
        )
    )


In [6]:
import json

with open("reconciliation_finetune.jsonl", "w") as f:
    for row in training_data:
        f.write(json.dumps(row) + "\n")


In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
import json
import os

# Configuration
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"  # Instruct version = better for tasks
OUTPUT_DIR = "./mistral_reconciliation_lora"
BATCH_SIZE = 2  # Reduced for memory
LEARNING_RATE = 5e-4
NUM_EPOCHS = 2

print(f"GPU Available: {torch.cuda.is_available()}")
print(f"GPU Count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

# ==================== FAST PATH: Load without Quantization ====================
print("Loading model in bfloat16 (FAST - no quantization overhead)...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,  # Native precision - much faster than quantization
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="flash_attention_2"  # Use flash attention for speed
)

# Enable gradient checkpointing to save memory
model.gradient_checkpointing_enable()

print("Model loaded successfully!")

# ==================== LoRA Configuration ====================
# LoRA adapters only fine-tune ~50M parameters out of 7B
lora_config = LoraConfig(
    r=8,                           # Smaller rank for speed
    lora_alpha=16,                 # Smaller alpha
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
print("\n========== LoRA Model Info ==========")
model.print_trainable_parameters()

# ==================== Tokenizer Setup ====================
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# ==================== Prepare Dataset ====================
print("Preparing dataset...")

def load_jsonl_dataset(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
    return data

raw_data = load_jsonl_dataset("reconciliation_finetune.jsonl")

# Create formatted text for training
formatted_data = []
for item in raw_data:
    messages = item["messages"]
    formatted_text = ""
    for msg in messages:
        formatted_text += f"{msg['role']}: {msg['content']}\n"
    formatted_data.append({"text": formatted_text})

# Save as temporary JSONL
with open("temp_formatted.jsonl", "w") as f:
    for item in formatted_data:
        f.write(json.dumps(item) + "\n")

# Load dataset
dataset = load_dataset("json", data_files="temp_formatted.jsonl")
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)

# Tokenize function
def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        padding="max_length",
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

print("Tokenizing dataset...")
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

print(f"Training samples: {len(tokenized_dataset['train'])}")
print(f"Validation samples: {len(tokenized_dataset['test'])}")

# ==================== Training Configuration ====================
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=8,
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_steps=50,
    weight_decay=0.01,
    optim="adamw_torch",  # Faster than paged_adamw
    logging_steps=5,
    save_steps=50,
    eval_steps=50,
    save_total_limit=2,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    report_to=["tensorboard"],
    logging_dir=f"{OUTPUT_DIR}/logs",
    bf16=True,  # Use bfloat16 for mixed precision
)

# ==================== Initialize Trainer ====================
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
)

# ==================== Start Fine-tuning ====================
print("\n" + "="*80)
print("Starting FAST Mistral LoRA fine-tuning (no quantization)...")
print("="*80)
print(f"Output directory: {OUTPUT_DIR}")
print(f"Training on {len(tokenized_dataset['train'])} samples\n")

trainer.train()

# ==================== Save Model ====================
print(f"\nSaving LoRA adapter to {OUTPUT_DIR}...")
model.save_pretrained(f"{OUTPUT_DIR}/adapter")
tokenizer.save_pretrained(f"{OUTPUT_DIR}/adapter")

print("✓ Fine-tuning complete!")
print(f"✓ LoRA adapter saved to {OUTPUT_DIR}/adapter")


GPU Available: False
GPU Count: 0
Loading model in bfloat16 (FAST - no quantization overhead)...


c:\Users\HarithaNagamalla\Documents\Python and ML\python_venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HarithaNagamalla\.cache\huggingface\hub\models--mistralai--Mistral-7B-Instruct-v0.1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
`torch_dtype` is deprecated! Use `dtype` instead!
Fetching 2 

In [ ]:
# ==================== Inference with Fine-tuned LoRA Model ====================
from peft import AutoPeftModelForCausalLM

print("Loading fine-tuned LoRA model for inference...")
model_inference = AutoPeftModelForCausalLM.from_pretrained(
    "./mistral_reconciliation_lora/adapter",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer_inference = AutoTokenizer.from_pretrained("./mistral_reconciliation_lora/adapter")

# Test prompt
test_prompt = """[INST] You are a financial reconciliation expert. Analyze these two transactions and determine if they match.

Ledger: Source=Bloomberg, Date=2022-05-07, Currency=CAD, Amount=-25633.43, Quantity=111, RefID=REF123
Statement: Source=Calypso, Date=2022-05-07, Currency=CAD, Amount=25633.43, Quantity=111, RefID=REF123

Return: matched (true/false) and confidence (0-1) [/INST]"""

print("\nGenerating prediction...")
inputs = tokenizer_inference(test_prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")

outputs = model_inference.generate(
    **inputs,
    max_new_tokens=200,
    do_sample=True,
    top_p=0.95,
    temperature=0.7,
)

response = tokenizer_inference.decode(outputs[0], skip_special_tokens=True)
print("\n" + "="*80)
print("MODEL PREDICTION:")
print("="*80)
print(response)


In [ ]:
# ==================== Step 2: QLoRA Configuration ====================
# LoRA adapters only fine-tune 1-2% of parameters
lora_config = LoraConfig(
    r=16,                          # LoRA rank
    lora_alpha=32,                 # LoRA alpha
    target_modules=["q_proj", "v_proj"],  # Target attention modules
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
# ==================== Step 3: Tokenizer Setup ====================
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# ==================== Step 4: Prepare Dataset ====================
print("Preparing dataset...")

# Load JSONL file
def load_jsonl_dataset(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
    return data

# Convert to dataset format
raw_data = load_jsonl_dataset("reconciliation_finetune.jsonl")

# Create formatted text for training
formatted_data = []
for item in raw_data:
    messages = item["messages"]
    formatted_text = ""
    for msg in messages:
        formatted_text += f"{msg['role']}: {msg['content']}\n"
    formatted_data.append({"text": formatted_text})

# Save as temporary JSONL for dataset loading
with open("temp_formatted.jsonl", "w") as f:
    for item in formatted_data:
        f.write(json.dumps(item) + "\n")

# Load dataset
dataset = load_dataset("json", data_files="temp_formatted.jsonl")

# Split into train and validation
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)

# Tokenize function
def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        padding="max_length",
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

print("Tokenizing dataset...")
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

print(f"Training samples: {len(tokenized_dataset['train'])}")
print(f"Validation samples: {len(tokenized_dataset['test'])}")



In [ ]:
# ==================== Step 5: Training Configuration ====================
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=4,
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_steps=100,
    weight_decay=0.01,
    optim="paged_adamw_8bit",  # Memory-efficient optimizer
    logging_steps=10,
    save_steps=50,
    eval_steps=50,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to=["tensorboard"],
    logging_dir=f"{OUTPUT_DIR}/logs",
    fp16=torch.cuda.is_available(),  # Use mixed precision if GPU available
)


In [ ]:
# ==================== Step 6: Initialize Trainer ====================
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
)

In [ ]:
# ==================== Step 7: Start Fine-tuning ====================
print("\nStarting fine-tuning with Mistral QLoRA...")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Training on {len(tokenized_dataset['train'])} samples\n")

trainer.train()

# ==================== Step 8: Save Model ====================
print(f"\nSaving model to {OUTPUT_DIR}...")
model.save_pretrained(f"{OUTPUT_DIR}/final_model")
tokenizer.save_pretrained(f"{OUTPUT_DIR}/final_model")

print("✓ Fine-tuning complete!")
print(f"✓ Model saved to {OUTPUT_DIR}/final_model")

In [ ]:
# ==================== Inference and Testing ====================
from peft import AutoPeftModelForCausalLM

# Load the fine-tuned model
MODEL_DIR = f"{OUTPUT_DIR}/final_model"

print("Loading fine-tuned model for inference...")
model_inference = AutoPeftModelForCausalLM.from_pretrained(
    MODEL_DIR,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer_inference = AutoTokenizer.from_pretrained(MODEL_DIR)
tokenizer_inference.pad_token = tokenizer_inference.eos_token

# Test with a sample reconciliation query
test_prompt = """You are a financial reconciliation expert.

Ledger Transaction:
{'Source': 'Bloomberg', 'ISIN_CUSIP': 'US0378331005', 'Trade_Date': '2022-05-07', 'Currency': 'CAD', 'Tran_code': 'DIV', 'Quantity': 111, 'signed_amount': -25633.43, 'Ref1': 'REF123', 'Ref2': 'REF456', 'Trade_status': 'SETTLED'}

Statement Transaction:
{'Source': 'Calypso', 'ISIN_CUSIP': 'US0378331005', 'Trade_Date': '2022-05-07', 'Currency': 'CAD', 'Tran_code': 'DIV', 'Quantity': 111, 'signed_amount': 25633.43, 'Ref1': 'REF123', 'Ref2': 'REF456', 'Trade_status': 'SETTLED'}

Task:
Check if these transactions reconcile.
Return:
- matched (true/false)
- reason
- confidence (0-1)"""

# Tokenize and generate
inputs = tokenizer_inference(test_prompt, return_tensors="pt").to("cuda")

outputs = model_inference.generate(
    **inputs,
    max_new_tokens=256,
    do_sample=True,
    top_p=0.95,
    top_k=40,
    temperature=0.7,
)

response = tokenizer_inference.decode(outputs[0], skip_special_tokens=True)
print("=" * 80)
print("MODEL RESPONSE:")
print("=" * 80)
print(response)


<!-- ## Mistral QLoRA Fine-tuning Summary

## What was done
✓ **4-bit Quantization**: Model compressed to 4-bit for GPU memory efficiency  
✓ **LoRA Adapters**: Fine-tuned only 1-2% of model parameters (~50M out of 7B)  
✓ **Memory-Efficient Training**: Used gradient checkpointing and paged optimizer  
✓ **Dataset Split**: 90% training, 10% validation on 903 reconciliation samples  
✓ **Training Config**: 3 epochs, batch size 4, learning rate 2e-4, cosine scheduler  

## Model Performance Metrics
- Monitor in tensorboard: `tensorboard --logdir ./mistral_reconciliation_qora/logs`
- Key metrics: Training loss, validation loss, perplexity
- Expected convergence: Loss should decrease steadily over epochs

## Output Structure
```
mistral_reconciliation_qora/
├── final_model/              # Fine-tuned weights + config
│   ├── adapter_config.json
│   ├── adapter_model.bin
│   ├── config.json
│   └── tokenizer_config.json
├── checkpoint-*/             # Intermediate checkpoints
└── logs/                      # TensorBoard logs
```

## Next Steps
1. **Inference**: Test the model on new reconciliation samples
2. **Evaluation**: Assess accuracy vs. baseline on held-out test set
3. **Deployment**: Merge adapters with base model for production
4. **Fine-tuning**: Add more data or adjust hyperparameters

## Merge Adapters with Base Model (Optional)
```python
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    "./mistral_reconciliation_qora/final_model",
    device_map="auto"
)
merged_model = model.merge_and_unload()
merged_model.save_pretrained("./mistral_merged")
``` -->
